# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # ignore notebook temp files
    # Code acquired from Jose G in Udacity Knowledge Base Article 47740
    if ('.ipynb_checkpoints' in root):
        continue
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of the project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster()

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkyfi_nosql
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkyfi_nosql')
except Exception as e:
    print(e)

### The first query in this project returns the artist, song title and song length based on the Session ID and Item in Session

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard \
## based on sessionId and itemInSession

# sessionId and itemInSession were selected as Primary Keys as those are what the main data are selected on
query = "CREATE TABLE IF NOT EXISTS artist_title_length "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# Code used for ingesting the data from the csv file for import into the Cassandra database #
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_title_length (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
## Use a SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM artist_title_length WHERE sessionId = 338 AND itemInSession = 4"

try:
    #rows = session.execute(query)
    artist_title_length_df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)

print(artist_title_length_df)
    
#for row in rows:
#    print(row.artist, row.song, row.length)

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


### The second query in this project returns Artist Name, Song Title sorted by Item in Session and User Name based on User ID and Session ID

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) \
## based on userid and sessionid.

# userId and sessionId are the composite primary key as those are what the data is selected on and itemInSession is the cluster key the data is sorted on
query2 = "CREATE TABLE IF NOT EXISTS artist_song_user "
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
                    PRIMARY KEY((userId, sessionId), itemInSession))"

try:
    session.execute(query2)
except Exception as e:
    print(e)

In [12]:
# Code used for ingesting the data from the csv file for import into the Cassandra database #
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query2 = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

        

In [13]:
## Use a SELECT statement to verify the data was entered into the table
query2 = "SELECT artist, song, firstName, lastName FROM artist_song_user WHERE userId = 10 AND sessionID = 182"

try:
    #rows = session.execute(query2)
    artist_song_user_df = pd.DataFrame(list(session.execute(query2)))
except Exception as e:
    print(e)

print(artist_song_user_df)

#for row in rows:
#    print(row.artist, row.song, row.firstname, row.lastname)

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


### The third query in this project returns User Names based on songs listened to

In [14]:
## Query 3: Give me every user name (first and last) in my music app history based on song title

# song and userId form a composite key to get all the user names of user who listen to a particular song
query3 = "CREATE TABLE IF NOT EXISTS user_song "
query3 = query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"

try:
    session.execute(query3)
except Exception as e:
    print(e)

                    

In [15]:
# Code used for ingesting the data from the csv file for import into the Cassandra database #
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query3 = "INSERT INTO user_song (song, userId, firstName, lastName)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

In [16]:
## Use a SELECT statement to verify the data was entered into the table
query3 = "SELECT firstName, lastName FROM user_song WHERE song = 'All Hands Against His Own'"

try:
    #rows = session.execute(query3)
    user_song_df = pd.DataFrame(list(session.execute(query3)))
except Exception as e:
    print(e)

print(user_song_df)

#for row in rows:
#    print(row.firstname, row.lastname)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


# Part III. Perform Clean Up

### Drop the tables before closing out the sessions

In [17]:
## Drop the tables before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS artist_title_length")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE IF EXISTS artist_song_user")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE IF EXISTS user_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()